In [ ]:
from faker import Faker
from pyspark.sql import SparkSession
from IPython.core.display import HTML

display(HTML("<style>pre { white-space: pre !important; }</style>"))
spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.read.csv('../Tbl_BusinessDoc.csv', header=True)

In [ ]:
def pretty_print(dataframe, max_lenght=20, **kwargs):
    import math
    max_lenght = max_lenght
    for col in dataframe.columns:
        if len(col) > max_lenght:
            max_lenght = len(col)
    for col in dataframe.columns:
        preffix = math.floor((max_lenght - len(col)) / 2)
        suffix = math.ceil((max_lenght - len(col)) / 2)
        new_name = " " * preffix + col + " " * suffix
        dataframe = dataframe.withColumnRenamed(col, new_name)
    dataframe.show(**kwargs)
    del(dataframe)

In [ ]:
def print_decorator(func):